<a href="https://colab.research.google.com/github/sobit-nep/Transformer-Neural-Network-from-scratch/blob/main/Self_Attention_in_transformer_neural_network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import math

L,d_k,d_v=4,8,8 #for four words sentence, eg. "My name is Shovit"
q = np.random.randn(L, d_k)
k = np.random.randn(L, d_k)
v = np.random.randn(L, d_v)

In [2]:
print("Q\n", q)
print("K\n", k)
print("V\n", v)

Q
 [[ 0.82072468 -0.08186315  0.39570796  0.64526011 -1.02650772  2.31904455
  -0.23446429  0.5405788 ]
 [-1.41997504  0.29517739 -1.85193499  1.52058291  1.33097357 -1.71247872
  -1.20118116 -0.0770579 ]
 [-0.11688462 -1.69123466  0.46714804  0.59474401 -2.19221197 -0.37400926
  -0.7655121   0.51208199]
 [ 0.1971251   0.76521849 -0.09746935  1.26936539 -1.50790003  0.74519826
   1.34664422  0.27103438]]
K
 [[ 0.72632534  0.48378084 -0.87949137  1.20783536 -0.47065671 -0.31492596
  -0.20685232 -1.44280894]
 [ 0.62062491  0.39095534  0.35662576  1.07046023 -0.43485367  0.34076939
  -0.08353512  0.48569724]
 [-0.45812393 -3.02665811  1.3440903   0.00491275 -0.43887945  0.64398229
  -1.09471788 -0.48741355]
 [ 2.20613891 -0.59654763 -0.37486852 -1.16204819  1.2727931  -0.47569271
   0.25422276  1.22490108]]
V
 [[ 0.09818446 -2.38935039  0.53703497  0.32469186 -0.82910017 -0.77045443
   2.1544596  -2.31736656]
 [-0.89201249  0.300139   -0.38043479  0.9463282   0.15840573 -0.23528902
  -0.1

SELF ATTENTION

          self attention = softmax((Q.K^T/sqrt(d_k))+M)V

In [3]:
np.matmul(q,k.T)

array([[ 0.00920851,  2.82798595,  2.34393896, -0.84582902],
       [ 2.84933128, -0.89801995, -3.05900347, -2.27259634],
       [-0.02650704,  1.20801256,  7.11283327, -2.29488157],
       [ 1.93769652,  2.67435728, -2.99575453, -3.05952171]])

In [4]:
# Why we need sqrt(d_k) in denominator
q.var(), k.var(), np.matmul(q, k.T).var()

(1.1946305514927924, 0.9727866453115241, 7.644810723906373)

In [5]:
scaled = np.matmul(q,k.T)/math.sqrt(d_k)
q.var(), k.var(), scaled.var()

(1.1946305514927924, 0.9727866453115241, 0.9556013404882964)

Notice the reduction in variance of product

In [6]:
scaled

array([[ 0.0032557 ,  0.99984402,  0.82870757, -0.29904572],
       [ 1.00739073, -0.317498  , -1.08152105, -0.80348414],
       [-0.00937166,  0.42709694,  2.51476632, -0.81136316],
       [ 0.68507918,  0.94552808, -1.05915917, -1.08170427]])

# **Masking**
• This is to ensure words don't get context from words generated in the future.



• Not required in the encoders, but required int he decoders

In [7]:
mask = np.tril(np.ones((L,L)))
mask

array([[1., 0., 0., 0.],
       [1., 1., 0., 0.],
       [1., 1., 1., 0.],
       [1., 1., 1., 1.]])

In [9]:
mask[mask==0]=-np.inf
mask[mask==1]=0

In [10]:
mask

array([[  0., -inf, -inf, -inf],
       [  0.,   0., -inf, -inf],
       [  0.,   0.,   0., -inf],
       [  0.,   0.,   0.,   0.]])

In [12]:
scaled+mask

array([[ 0.0032557 ,        -inf,        -inf,        -inf],
       [ 1.00739073, -0.317498  ,        -inf,        -inf],
       [-0.00937166,  0.42709694,  2.51476632,        -inf],
       [ 0.68507918,  0.94552808, -1.05915917, -1.08170427]])

# **SOFTMAX**
    SOFTMAX(x_i)=exp(x_i)/sum(exp(x_j))

In [13]:
def softmax(x):
  return (np.exp(x). T /np.sum (np.exp(x), axis=-1)).T

In [15]:
attention = softmax(scaled + mask)

In [16]:
attention

array([[1.        , 0.        , 0.        , 0.        ],
       [0.78999391, 0.21000609, 0.        , 0.        ],
       [0.06654526, 0.10296107, 0.83049367, 0.        ],
       [0.37833327, 0.49089212, 0.06612435, 0.06465025]])

In [18]:
v_new = np.matmul(attention, v)
v_new

array([[ 0.09818446, -2.38935039,  0.53703497,  0.32469186, -0.82910017,
        -0.77045443,  2.1544596 , -2.31736656],
       [-0.10976293, -1.82454125,  0.34436074,  0.45523928, -0.62171792,
        -0.65806643,  1.67529117, -1.69399501],
       [ 1.02784612,  1.44880678, -0.17450204,  0.33126458, -0.21698996,
        -0.79134061, -0.90025442, -0.86934166],
       [-0.29115397, -0.55633001, -0.0302895 ,  0.65806338, -0.2747558 ,
        -0.52275294,  0.78807835, -0.60709351]])

In [19]:
v

array([[ 0.09818446, -2.38935039,  0.53703497,  0.32469186, -0.82910017,
        -0.77045443,  2.1544596 , -2.31736656],
       [-0.89201249,  0.300139  , -0.38043479,  0.9463282 ,  0.15840573,
        -0.23528902, -0.12722867,  0.65098337],
       [ 1.34035334,  1.8987551 , -0.20598495,  0.25553826, -0.21448314,
        -0.86195107, -1.24085701, -0.94179867],
       [ 0.32407423,  1.1562355 , -0.51190265,  0.83184956, -0.38139337,
        -0.90899741,  1.81717798,  0.19112976]])